In [1]:
## ---------------------------------------------------------------------
## set up configs for huggingface hub and OS paths on HPC cluster -- make sure config.ini is correct
## ---------------------------------------------------------------------
import configparser

def scratch_path():
    config = configparser.ConfigParser()
    config.read("config.ini")
    return "/scratch/" + config["user"]["username"]

import os
if os.path.isdir(scratch_path()):
    os.environ['TRANSFORMERS_CACHE'] = scratch_path() + '/.cache/huggingface'
    os.environ['HF_DATASETS_CACHE'] = scratch_path() + '/.cache/huggingface/datasets'
print(os.getenv('TRANSFORMERS_CACHE'))
print(os.getenv('HF_DATASETS_CACHE'))

## ---------------------------------------------------------------------
## Load libraries
## ---------------------------------------------------------------------

import numpy as np
import pandas as pd

import torch
import transformers
from transformers import AutoTokenizer, AutoModel, LlamaForCausalLM, LlamaTokenizer, AutoModelForCausalLM

import torch.nn.functional as F

from baukit import Trace

from steering import *
## ---------------------------------------------------------------------
## Ensure GPU is available -- device should == 'cuda'
## ---------------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/scratch/dmpowell/.cache/huggingface
/scratch/dmpowell/.cache/huggingface/datasets


/home/dmpowell/.conda/envs/py3.11transformers4.44/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


device =  cuda


In [2]:
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_NAME = "meta-llama/Llama-3.1-8B"

wmodel = SteeringModel(
    AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,  # Replace this with the 70B variant if available
        torch_dtype=torch.bfloat16,
        device_map=device  # Automatically distributes the model across available GPUs
    ),
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, device = 'cuda', use_fast = False)
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Multiple choice

Here is a basic implementation of multiple choice answering using "cloze" probabilities. This should roughly work with both raw and instruction-tuned models.

In [5]:
import re

def answer_choice_list(choices):
    options = re.split(r'\s*\(\w\)\s*', choices)
    return( [option.strip() for option in options if option] )


def format_question(question):
    return f"Q: {question}\nA:"


# def format_statement(question, choices):
#     choice_string = ", ".join(choices)
#     return f"Please rate your agreement with the following statement, using the following scale: [{choice_string}]. Statement: {question}\nResponse:"


def format_with_instructions(instruction, question, choices):
    # choice_string = "; ".join(choices)
    return f"{instruction} Specifically, please use the following response options: {choices}.\n\nStatement: {question}\nResponse:"


def format_with_mcqa_instructions(instruction, question, choices_text):
    
    LETTERS = [chr(i) for i in range(65,91)]
    choices = re.split(';\W', choices_text)
    choices = [c.strip() for c in choices]
    labeled_choices = [". ".join([a,b]) for a, b in zip(LETTERS, choices)]
    labeled_choices = "\n".join(labeled_choices)
    
    return f"{instruction} Respond with the letter corresponding to your choice from the following response options:\n\n{labeled_choices}\n\nStatement: {question}\nResponse:"


def format_chat_question(instruction, question, choices):
    return f"{instruction} Specifically, please use the following response options: {choices}.\n\nStatement: {question}"


def format_mcqa_chat_question(instruction, question, choices_text):
    
    LETTERS = [chr(i) for i in range(65,91)]
    choices = re.split(';\W', choices_text)
    choices = [c.strip() for c in choices]
    labeled_choices = [". ".join([a,b]) for a, b in zip(LETTERS, choices)]
    labeled_choices = "\n".join(labeled_choices)
    
    return f"{instruction} Respond with the letter corresponding to your choice from the following response options:\n\n{labeled_choices}\n\nStatement: {question}"


def format_chat(instruction, question, choices):

    chat = [
        {"role": "user", "content": format_chat_question(instruction, question, choices)},
        {"role": "system", "content": "My Response:"}
    ]

    tokens = wmodel.tok.apply_chat_template(chat, tokenize=True, continue_final_message=True)[:-1]

    return(wmodel.tok.decode(tokens))


def format_mcqa_chat(instruction, question, choices):

    chat = [
        {"role": "user", "content": format_mcqa_chat_question(instruction, question, choices)},
        {"role": "system", "content": "My Response:"}
    ]

    tokens = wmodel.tok.apply_chat_template(chat, tokenize=True, continue_final_message=True)[:-1]

    return(wmodel.tok.decode(tokens))


def mc_choice_probs(model, question, choices, pad = True):
    prompt = question
    if pad:
        choices = [" " + c for c in choices] # pad all the 
    
    prompts = [prompt for c in choices]
    
    logits = torch.tensor([model.completion_logprob(x[0], x[1]) for x in zip(prompts, choices)])
    
    return(F.log_softmax(logits, -1).exp())


def choice_score(choice_probs):
    # calculate score on -1 to 1 scale
    choice_score01 = choice_probs @ torch.arange(len(choice_probs), dtype = choice_probs.dtype)/(len(choice_probs)-1)
    return (choice_score01.item() - .5)*2


def get_mean_steering_vector(ziplist, model, layer_id = 15):
    return torch.concat([model.get_steering_vector(s1, s2, layer_id) for s1, s2 in ziplist]).mean(0).unsqueeze(0)


def act_add(steering_vec):
    def act_add(output):
        z = torch.ones_like(output[0])
        z = z * steering_vec
        out = output[0] + z
        return (out, output[1])
    return act_add

For any agree/disagree etc. style scales, we can take the choice probabilities and compute a "score". I noticed the model seems to have a really strong "agree" bias when we have a pure "agree" option. Will need to look into this, probably some literature on it.

In [7]:
# choice_text = ['Strongly disagree', 'Somewhat disagree', "Neither agree nor disagree", 'Somewhat agree', 'Strongly agree']
# # q = format_question('Slavery benefitted the slaves, many of whom learned valuable skills.')
# q = format_statement('Slavery benefitted the slaves, many of whom learned valuable skills.', choice_text)
# choice_probs = mc_choice_probs(wmodel, q, choice_text )
# choice_score(choice_probs), choice_probs

## Steering

Applying a steering vector shifts generations ...

In [8]:



steering_vec = get_mean_steering_vector([["Love", "Hate"], ["Beautiful", "Disgusting"], ["Loving", "Hateful"]], wmodel)

test_sentence = "In my opinion, dogs"

print(wmodel.generate(test_sentence, max_new_tokens = 20))
print("-"*20)
with Trace(wmodel.get_module(), edit_output = act_add(steering_vec)):
    print(wmodel.generate(test_sentence, max_new_tokens = 20))
print("-"*20)
with Trace(wmodel.get_module(), edit_output = act_add(-1.5*steering_vec)):
    print(wmodel.generate(test_sentence, max_new_tokens = 20))

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


['In my opinion, dogs are the most wonderful creatures on this earth. They are loyal, loving, and always happy to see']
--------------------
['In my opinion, dogs are one of the most loyal and loving animals on the planet. They are a true companion and are']
--------------------
['In my opinion, dogs are the most disgusting things in the world. They are disgusting, filthy, and dise-ridden.']


### "survey" responses and attitude scores

Can apply the same thing to our multiple choice decisions and calculated scores. Adding a relevant steering vector affects the responses.

In [19]:
steering_vec = wmodel.get_steering_vector("Slavery is a proud part of American history.", "Slavery is the most shameful part of America's past.")

# steering_vec = get_mean_steering_vector([["Some races are inferior to others.", "No race is inferior to others."], ["White people are superior to black people.", "White and black people are equally good."]], wmodel)

# q = format_question('Slavery benefitted the slaves, many of whom learned valuable skills.')
q = format_statement('Slavery benefitted the slaves, many of whom learned valuable skills.', choice_text)

with Trace(wmodel.get_module(), edit_output = act_add(steering_vec)):
    # print(wmodel.generate(test_sentence, max_new_tokens = 20))
    choice_probs = mc_choice_probs(wmodel, format_question('Slavery benefitted the slaves who learned valuable skills.'), ['Strongly disagree',  'Somewhat disagree', "Neither agree nor disagree", 'Somewhat agree', 'Strongly agree'])
    print(choice_score(choice_probs), choice_probs) 

with Trace(wmodel.get_module(), edit_output = act_add(-steering_vec)):
    # print(wmodel.generate(test_sentence, max_new_tokens = 20))
    choice_probs = mc_choice_probs(wmodel, format_question('Slavery benefitted the slaves who learned valuable skills.'), ['Strongly disagree',  'Somewhat disagree', "Neither agree nor disagree", 'Somewhat agree', 'Strongly agree'])
    print(choice_score(choice_probs), choice_probs) 

0.0900799036026001 tensor([0.4413, 0.0123, 0.0035, 0.0108, 0.5321])
-0.8198438286781311 tensor([0.9074, 0.0010, 0.0026, 0.0018, 0.0872])


## Applying to survey ideology scales

First, to generate the model's answers.

I think the overall wisest way to go about things is with MCQA wih the instruct model. But interesting also with the non-instruct model, but we need a good few-shot prompt for the MCQA behavior. 

In [3]:
scales = pd.read_csv("data/scales.tsv", sep="\t")

scales = scales.loc[lambda x: x.sub_scale != 'not scored']
scales['response_options'] = [re.sub(r"\s*\(.*?\)\s*", " ", text).strip() for text in scales['response_options']]
scales['statement'] = [text.strip() for text in scales['statement']]
scales['simple_contrastive_statement'] = [text.strip() for text in scales['simple_contrastive_statement']]

scales['statement'] = [text + "." if text[-1]!="." else text for text in scales['statement']]
scales['simple_contrastive_statement'] = [text + "." if text[-1]!="." else text for text in scales['simple_contrastive_statement'] ]

0      (1) Very negative; (2) Negative; (3) Slightly ...
1      (1) Very negative; (2) Negative; (3) Slightly ...
2      (1) Very negative; (2) Negative; (3) Slightly ...
3      (1) Very negative; (2) Negative; (3) Slightly ...
4      (1) Very negative; (2) Negative; (3) Slightly ...
                             ...                        
265    (1) Strongly disagree; (2) Disagree; (3) Neutr...
266    (1) Strongly disagree; (2) Disagree; (3) Neutr...
267    (1) Strongly disagree; (2) Disagree; (3) Neutr...
268    (1) Strongly disagree; (2) Disagree; (3) Neutr...
269    (1) Strongly disagree; (2) Disagree; (3) Neutr...
Name: response_options, Length: 270, dtype: object

In [11]:
## Getting model responses
MCQA = True
resps = []
resp_probs = []

for idx, row in scales.iterrows():

    if MCQA:
        LETTERS = [chr(i) for i in range(65,91)]
        choices = re.split(';\W', row['response_options'])
        choices = LETTERS[:len(choices)]
        
        if MODEL_NAME=="meta-llama/Llama-3.1-8B":
            q = format_with_mcqa_instructions(row['instruction'], row['question'], row['response_options'])
        elif MODEL_NAME=="meta-llama/Llama-3.1-8B-Instruct":
            q = format_with_mcqa_instructions(row['instruction'], row['question'], row['response_options']) # format_mcqa_chat
    
    else:
        choices = re.split(';\W', row['response_options'])
        choices = [c.strip() for c in choices]

        if MODEL_NAME=="meta-llama/Llama-3.1-8B":
            q = format_with_instructions(row['instruction'], row['question'], row['response_options'])
        elif MODEL_NAME=="meta-llama/Llama-3.1-8B-Instruct":
            q = format_chat(row['instruction'], row['question'], row['response_options'])
    
    choice_probs = mc_choice_probs(wmodel, q, choices) # format_chat for instruct model

    resp_probs.append(choice_probs.detach().numpy())
    resps.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))
    
scales["response_probs"] = resp_probs
scales["model_score"] = resps

scales.groupby(['scale', 'sub_scale']).agg(avg_score = ('model_score', 'mean'))

KeyboardInterrupt: 

In [15]:
scales.query('sub_scale == "Harm"')

,item,instruction,original_instruction,question,statement,simple_contrastive_statement,strong_constrastive_statement,response_options,scale,construct,original_item,direction,sub_scale,explicit_social_bias,response_probs,model_score
16,17,When you decide whether something is right or ...,True,Whether or not someone suffered emotionally,I care a lot about someone suffering emotionally.,I don't care at all about someone suffering em...,Whether or not someone suffered emotionally ha...,Not at all relevant; Not very relevant; Slight...,MFQ,Moral Foundations,True,high,Harm,FALSE,"[0.01597328, 0.11802746, 0.2205043, 0.32083187...",0.169737
21,23,When you decide whether something is right or ...,True,Whether or not someone cared for someone weak ...,I care a lot about caring for the weak and vul...,I don't care at all about caring for the weak ...,Whether or not someone cared for someone weak ...,Not at all relevant; Not very relevant; Slight...,MFQ,Moral Foundations,True,high,Harm,FALSE,"[0.011301245, 0.050648663, 0.12149983, 0.33027...",0.354235
26,28,When you decide whether something is right or ...,True,Whether or not someone was cruel,I would want to stop someone who was being cruel.,I would have no problem with someone who was b...,Whether or not someone was cruel has nothing t...,Not at all relevant; Not very relevant; Slight...,MFQ,Moral Foundations,True,high,Harm,FALSE,"[0.018671598, 0.047679596, 0.07861036, 0.21368...",0.461402
31,33,Please read the following sentences and indica...,True,Compassion for those who are suffering is the ...,Compassion for those who are suffering is the ...,Compassion for those who are suffering is not ...,Compassion for those who are suffering is the ...,Strongly disagree; Moderately disagree; Slight...,MFQ,Moral Foundations,True,high,Harm,FALSE,"[0.028862413, 0.07845617, 0.041994564, 0.08890...",0.571110
36,39,Please read the following sentences and indica...,True,One of the worst things a person could do is h...,One of the worst things a person could do is h...,Hurting a defenseless animal is not one of the...,One of the best things a person could do is hu...,Strongly disagree; Moderately disagree; Slight...,MFQ,Moral Foundations,True,high,Harm,FALSE,"[0.008144333, 0.0055975127, 0.0020592099, 0.00...",0.952610
41,44,Please read the following sentences and indica...,True,It can never be right to kill a human being.,It can never be right to kill a human being.,It can sometimes be right to kill a human being.,It is always right to kill a human being.,Strongly disagree; Moderately disagree; Slight...,MFQ,Moral Foundations,True,high,Harm,FALSE,"[0.044025667, 0.02356524, 0.005258116, 0.01261...",0.845741


## quick steering test


How should we measure "opinions"?

- OpinionQA paper (https://arxiv.org/pdf/2303.17548) used MCQA
- Steering language models paper used MCQA

So probably need to use MCQA

In [63]:
def make_mcqa_steering_statements(statement, response_options):
    choices = re.split(';\W', response_options)
    choices = [choices[0], choices[-1]]
    prompts = [f"{statement}\n\nWhat do you think? A) {choices[0]} B) {choices[1]}\n\nAnswer: {choice}" for choice in ['A', 'B']]
    return(prompts)


def make_mcqa_steering_from_items(items):
    statements = items.statement.to_list()
    directions = items.direction.to_list()
    response_options = items.response_options.to_list()
    steering_statements = [make_mcqa_steering_statements(s, r) for s, r in zip(statements, response_options)]
    steering_statements = [[s[1], s[0]] if d == "low" else s for s, d in zip(steering_statements, directions)]

    return(steering_statements)


make_mcqa_steering_from_items(items)



[['Threatening a partner is okay as long as I don’t hurt him or her.\n\nWhat do you think? A) Strongly disagree B) Strongly Agree\n\nAnswer: A',
  'Threatening a partner is okay as long as I don’t hurt him or her.\n\nWhat do you think? A) Strongly disagree B) Strongly Agree\n\nAnswer: B'],
 ['During a heated argument, it is okay for me to bring up something from my partner’s past to hurt him or her.\n\nWhat do you think? A) Strongly disagree B) Strongly Agree\n\nAnswer: A',
  'During a heated argument, it is okay for me to bring up something from my partner’s past to hurt him or her.\n\nWhat do you think? A) Strongly disagree B) Strongly Agree\n\nAnswer: B'],
 ['As long as my partner doesn’t hurt me, “threats” are excused.\n\nWhat do you think? A) Strongly disagree B) Strongly Agree\n\nAnswer: A',
  'As long as my partner doesn’t hurt me, “threats” are excused.\n\nWhat do you think? A) Strongly disagree B) Strongly Agree\n\nAnswer: B'],
 ['During a heated argument, it is okay for me to

In [68]:
# sdo = scales.loc[lambda x: ((x.scale == "SJS") & (x.direction == 'high'))]
# sdo_zipped = zip(sdo.statement.to_list(), sdo.simple_contrastive_statement.to_list())
# steering_vec = get_mean_steering_vector(sdo_zipped, wmodel)


## Getting model responses
MCQA = True
resps = []
resp_probs = []
resps_posvec = []
resp_probs_posvec = []
resps_negvec = []
resp_probs_negvec = []

curr_subscale = ""

for idx, row in scales.iterrows():
    if row['sub_scale'] != curr_subscale:
        curr_subscale = row['sub_scale']
        # items = scales.loc[lambda x: ((x.sub_scale == curr_subscale) & (x.direction == 'high'))]
        items = scales.loc[lambda x: ((x.sub_scale == curr_subscale))]
        # if len(items) == 0:
        #     items = scales.loc[lambda x: ((x.sub_scale == curr_subscale) & (x.direction == 'low'))]
        #     items_zipped = zip(items.simple_contrastive_statement.to_list(), items.statement.to_list())
        # else:
        #     items_zipped = zip(items.statement.to_list(), items.simple_contrastive_statement.to_list())
        items_zipped = make_mcqa_steering_from_items(items)

        steering_vec = get_mean_steering_vector(items_zipped, wmodel, 12)

    if MCQA:
        LETTERS = [chr(i) for i in range(65,91)]
        choices = re.split(';\W', row['response_options'])
        choices = LETTERS[:len(choices)]
        
        if MODEL_NAME=="meta-llama/Llama-3.1-8B":
            q = format_with_mcqa_instructions(row['instruction'], row['question'], row['response_options'])
        elif MODEL_NAME=="meta-llama/Llama-3.1-8B-Instruct":
            q = format_mcqa_chat(row['instruction'], row['question'], row['response_options'])
    
    else:
        choices = re.split(';\W', row['response_options'])
        choices = [c.strip() for c in choices]

        if MODEL_NAME=="meta-llama/Llama-3.1-8B":
            q = format_with_instructions(row['instruction'], row['question'], row['response_options'])
        elif MODEL_NAME=="meta-llama/Llama-3.1-8B-Instruct":
            q = format_chat(row['instruction'], row['question'], row['response_options'])

    choice_probs = mc_choice_probs(wmodel, q, choices) # format_chat for instruct model

    resp_probs.append(choice_probs.detach().numpy())
    resps.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))
    
    with Trace(wmodel.get_module(), edit_output = act_add(steering_vec)):
        choice_probs = mc_choice_probs(wmodel, q, choices) # format_chat for instruct model

    resp_probs_posvec.append(choice_probs.detach().numpy())
    resps_posvec.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))

    with Trace(wmodel.get_module(), edit_output = act_add(-steering_vec)):
        choice_probs = mc_choice_probs(wmodel, q, choices) # format_chat for instruct model

    resp_probs_negvec.append(choice_probs.detach().numpy())
    resps_negvec.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))
    
scales["response_probs"] = resp_probs
scales["model_score"] = resps

scales["response_probs_posvec"] = resp_probs_posvec
scales["model_score_posvec"] = resps_posvec

scales["response_probs_negvec"] = resp_probs_negvec
scales["model_score_negvec"] = resps_negvec

In [70]:
(
    scales
    .groupby(['scale', 'sub_scale'])
    .agg(
        avg_score = ('model_score', 'mean'),
        avg_pos = ('model_score_posvec', 'mean'),
        avg_neg = ('model_score_negvec', 'mean')
    )
    .assign(
        coherent = lambda d: d.apply(lambda x: (x.avg_pos > x.avg_score) & (x.avg_neg < x.avg_score), 1)
        )
    # .query('direction == "high"')
)

avg_score   avg_pos   avg_neg  coherent
scale sub_scale                                                              
CSES  Importance to Identity           0.218305  0.137398  0.234458     False
      Membership self-esteem.          0.146098  0.088890  0.155281     False
      Private collective self-esteem   0.113998  0.089829  0.119867     False
      Public collective self-esteem    0.117354  0.093614  0.102778     False
CVS   Capitalistic Values              0.154730  0.085871  0.152869     False
GENE  Generalized Ethnocentrism       -0.157749 -0.148879 -0.165248      True
IPVAS Control                         -0.135758 -0.121896 -0.146690      True
      Threat                          -0.337151 -0.283281 -0.249659     False
      Violence                        -0.583751 -0.148060  0.172154     False
JWS   Just World Belief                0.060980  0.062948  0.049498      True
LWAI  Anticonventionalism             -0.256659 -0.053944 -0.268281      True
      Antihierarchical Aggression     -0.364906 -0.186848 -0.482429      True
      Top-Down Censorship             -0.007525 -0.022605 -0.000129     False
MFQ   Authority                        0.072525 -0.027465 -0.121567     False
      Fairness                         0.600666  0.342188  0.380271     False
      Harm                             0.559139  0.338438  0.298065     False
      Ingroup                          0.172293 -0.037015  0.017763     False
      Purity                          -0.009540 -0.062734 -0.115011     False
PECS  Politico-Economic Conservatism   0.001948 -0.064346  0.049387     False
RFS   Religious Fundamentalism        -0.179916 -0.158744 -0.198222      True
RWAS  Religious Fundamentalism        -0.200204 -0.193014 -0.199776     False
SDO-7 antiegalitarianism              -0.413235 -0.392073 -0.425529      True
      dominance                       -0.446201 -0.447124 -0.435669     False
SJS   Just World Belief               -0.029829 -0.065567 -0.001346     False

In [54]:
steering_vec = get_mean_steering_vector([
                                        #  ["The world is basically a just place.", "The world is basically an unjust place."], 
                                         ["I know good deeds rarely go unnoticed and unrewarded.", "I know good deeds often go unnoticed and unrewarded."],
                                         ], wmodel)

# steering_vec = wmodel.get_steering_vector("Agree", "Disagree")

def get_model_answers(d, wmodel, steering_vec, scale_factor = 1, MCQA=True):
    
    resps = []
    resp_probs = []
    resps_posvec = []
    resp_probs_posvec = []
    resps_negvec = []
    resp_probs_negvec = []

    for idx, row in d.iterrows():

        if MCQA:
            LETTERS = [chr(i) for i in range(65,91)]
            choices = re.split(';\W', row['response_options'])
            choices = LETTERS[:len(choices)]
            
            if MODEL_NAME=="meta-llama/Llama-3.1-8B":
                q = format_with_mcqa_instructions(row['instruction'], row['question'], row['response_options'])
            elif MODEL_NAME=="meta-llama/Llama-3.1-8B-Instruct":
                q = format_mcqa_chat(row['instruction'], row['question'], row['response_options'])
        
        else:
            choices = re.split(';\W', row['response_options'])
            choices = [c.strip() for c in choices]

            if MODEL_NAME=="meta-llama/Llama-3.1-8B":
                q = format_with_instructions(row['instruction'], row['question'], row['response_options'])
            elif MODEL_NAME=="meta-llama/Llama-3.1-8B-Instruct":
                q = format_chat(row['instruction'], row['question'], row['response_options'])

        choice_probs = mc_choice_probs(wmodel, q, choices) # format_chat for instruct model

        resp_probs.append(choice_probs.detach().numpy())
        resps.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))
        
        with Trace(wmodel.get_module(), edit_output = act_add(scale_factor*steering_vec)):
            choice_probs = mc_choice_probs(wmodel, q, choices) # format_chat for instruct model

        resp_probs_posvec.append(choice_probs.detach().numpy())
        resps_posvec.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))

        with Trace(wmodel.get_module(), edit_output = act_add(-scale_factor*steering_vec)):
            choice_probs = mc_choice_probs(wmodel, q, choices) # format_chat for instruct model

        resp_probs_negvec.append(choice_probs.detach().numpy())
        resps_negvec.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))
        
    d["response_probs"] = resp_probs
    d["model_score"] = resps

    d["response_probs_posvec"] = resp_probs_posvec
    d["model_score_posvec"] = resps_posvec

    d["response_probs_negvec"] = resp_probs_negvec
    d["model_score_negvec"] = resps_negvec

    return(d)


df = scales.query('scale == "JWS"')
res = get_model_answers(df, wmodel, steering_vec, .25)

res
(
    res
    .groupby(['scale', 'direction'])
    .agg(
        avg_score = ('model_score', 'mean'),
        avg_pos = ('model_score_posvec', 'mean'),
        avg_neg = ('model_score_negvec', 'mean')
    )
    .assign(
        coherent = lambda d: d.apply(lambda x: (x.avg_pos > x.avg_score) & (x.avg_neg < x.avg_score), 1)
        )
)

/tmp/ipykernel_1496006/1494890684.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["response_probs"] = resp_probs
/tmp/ipykernel_1496006/1494890684.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["model_score"] = resps
/tmp/ipykernel_1496006/1494890684.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

avg_score   avg_pos   avg_neg  coherent
scale direction                                         
JWS   high        0.062114  0.261401 -0.090248      True
      low         0.059594 -0.181123  0.219260     False

0.7999999523162842